In [1]:
%load_ext autoreload
%autoreload 2

In [94]:
from absbox.local.generic import Generic

test01 = Generic(
    "TEST01"
    ,{"collect":["2022-05-01","2022-06-01"] 
      ,"pay":["2022-06-15","2022-07-15"] # next distribution date
      ,"payFreq":["DayOfMonth",20]
      ,"poolFreq":"MonthEnd"
      ,"stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]
      ,'issuanceStat':{"IssuanceBalance":10000}
     }
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,{"AfterCollect":{
        "DefaultTrigger": 
         {"condition":[("poolBalance",),"<",354.03]
          ,"effects":("newStatus","Accelerated")
          ,"status":False
          ,"curable":False}}}
    ,"Amortizing"
    )

In [95]:
from absbox import API
localAPI = API("http://localhost:8081",lang='english',check=False)
#localAPI = API("https://absbox.org/api/dev",lang='english',check=False)

✅Connected, local lib:0.20.2, server:0.21.13

In [96]:
r = localAPI.run(test01
               ,poolAssump = ("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                       ,None
                                       ,None)
               ,runAssump = [("inspect",["MonthEnd",("poolFactor",)])]
               ,read=True)

In [97]:
r['result']['status']

,Date,From,To
0,2023-05-31,Amortizing,Accelerated


In [98]:
r['result']['logs']

,Type,Comment
0,Error,No waterfall distribution found on date2023-06...
1,Error,No waterfall distribution found on date2023-07...
2,Error,No waterfall distribution found on date2023-08...
3,Error,No waterfall distribution found on date2023-09...
4,Error,No waterfall distribution found on date2023-10...
...,...,...
76,Error,No waterfall distribution found on date2029-10...
77,Error,No waterfall distribution found on date2029-11...
78,Error,No waterfall distribution found on date2029-12...
79,Warning,Bond B is not paid off


In [87]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-05-01,1777.15,0.00,0.00,0,0.00,0,0.00,0.00,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-01,1562.58,211.69,22.95,0,2.88,0,2.88,0.08,None,None,211.69,0.0,0.0,2.88,0.0,2.88
2022-07-31,1454.52,106.77,10.40,0,1.29,0,1.29,0.08,None,None,318.46,0.0,0.0,4.17,0.0,4.17
2022-08-31,1345.89,107.39,9.68,0,1.24,0,1.24,0.08,None,None,425.85,0.0,0.0,5.41,0.0,5.41
2022-09-30,1236.74,108.01,8.96,0,1.14,0,1.14,0.08,None,None,533.86,0.0,0.0,6.55,0.0,6.55
2022-10-31,1127.07,108.65,8.23,0,1.02,0,1.02,0.08,None,None,642.51,0.0,0.0,7.57,0.0,7.57
2022-11-30,1016.83,109.28,7.50,0,0.96,0,0.96,0.08,None,None,751.79,0.0,0.0,8.53,0.0,8.53
2022-12-31,906.09,109.91,6.77,0,0.83,0,0.83,0.08,None,None,861.70,0.0,0.0,9.36,0.0,9.36
2023-01-31,794.77,110.55,6.03,0,0.77,0,0.77,0.08,None,None,972.25,0.0,0.0,10.13,0.0,10.13


In [93]:
r['result']['inspect']

{'<PoolFactor>':             <PoolFactor>
 Date                    
 2022-05-01      0.000000
 2022-05-31      0.000000
 2022-06-30      0.156258
 2022-07-31      0.145452
 2022-08-31      0.134589
 ...                  ...
 2029-08-31      0.000000
 2029-09-30      0.000000
 2029-10-31      0.000000
 2029-11-30      0.000000
 2029-12-31      0.000000
 
 [93 rows x 1 columns]}

In [41]:
r['pool']['flow']['Principal'].sum() + r['pool']['flow']['Default'].sum()

1777.15

In [ ]:
r['pool']['flow']['Principal']

In [37]:
1764.42+115.17

1879.5900000000001

In [38]:
1562.58+211.69

1774.27

In [39]:
1777.15 - 1764.42

12.730000000000018